# Mining Big Datasets | Assignment 1 | Full Time 2021-2022



* ### John Vaniotis | f2822101
* ### Orestis Loukopoulos | f2822104

In [1]:
# Import the necessary libraries

import pandas as pd
import numpy as np
import itertools
import copy

### 1) Import and pre-process the dataset with customers
Download the bank.csv dataset from moodle. This dataset is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls to access the opinion of the customer to the bank service and products. The dataset includes 43191 bank customer profiles with 10 attributes each. Below is a description of the available attributes:


* <b>Age</b>: The age of the customer.
* <b>Job</b>: type of job (admin, unknown, unemployed, management, housemaid, entrepreneur, student, blue-collar, self-employed, retired, technician, services).
* <b>Marital Status</b>: Married, Single, Divorced.
* <b>Education</b>: Primary, Secondary, Tertiary.
* <b>Default</b>: If the customer has credit in default (yes/no). Balance: average yearly balance, in euros.
* <b>Housing</b>: if the customer has a housing loan (yes/no). Loan: if the customer has a personal loan (yes/no)
* <b>Customer Rating</b>: The rating of the bank from the customer (Poor, Fair, Good, Very Good, Excellent).
* <b>Products</b>: An array containing the bank products (1-20) each customer has.


For any numerical values missing, you should replace them with the average value of the attribute in the dataset (rounded to the nearest integer). The replaced average values calculated should be reported in the pdf.

* First we have to import the `bank` dataset.

In [2]:
bank = pd.read_csv('bank.csv', sep=';')
bank

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
0,33.0,entrepreneur,married,secondary,no,2,yes,yes,poor,"1,3,16,17,19"
1,35.0,management,married,tertiary,no,231,yes,no,good,"4,8,16"
2,NaN,management,single,tertiary,no,447,yes,yes,fair,"7,16"
3,42.0,entrepreneur,divorced,tertiary,yes,2,yes,no,fair,"1,3,8,10,11,12,18,19"
4,58.0,retired,married,primary,no,121,yes,no,good,"4,5,6,7,11,18,19"
...,...,...,...,...,...,...,...,...,...,...
43186,51.0,technician,married,tertiary,no,825,no,no,very_good,"12,15"
43187,71.0,retired,divorced,primary,no,1729,no,no,very_good,"4,17,19"
43188,72.0,retired,married,secondary,no,5715,no,no,very_good,"5,7,8,9,10,11,13,16,17,18"
43189,57.0,blue-collar,married,secondary,no,668,no,no,very_good,"2,3,7"


* Check for NAs in the dataset.

In [3]:
bank.isna().sum()

Age          1000
Job             0
Marital         0
Education       0
Default         0
Balance         0
Housing         0
Loan            0
Rating          0
Products        0
dtype: int64

* We can see that the attribute `Age` contains 1000 NAs. We will replace them with the average value of the attribute `Age`.
* We calculated the average of the column `Age` and we rounded it to the nearest integer. Therefore, the mean Age of a Customer in our dataset is 41 years old.

In [4]:
#Replace NAs in column Age with the average value of the attribute Age
average_age = round(bank.Age.mean())
bank.Age.loc[bank.Age.isna()==True] = average_age
bank.isna().sum()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Age          0
Job          0
Marital      0
Education    0
Default      0
Balance      0
Housing      0
Loan         0
Rating       0
Products     0
dtype: int64

In [5]:
bank

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
0,33.0,entrepreneur,married,secondary,no,2,yes,yes,poor,"1,3,16,17,19"
1,35.0,management,married,tertiary,no,231,yes,no,good,"4,8,16"
2,41.0,management,single,tertiary,no,447,yes,yes,fair,"7,16"
3,42.0,entrepreneur,divorced,tertiary,yes,2,yes,no,fair,"1,3,8,10,11,12,18,19"
4,58.0,retired,married,primary,no,121,yes,no,good,"4,5,6,7,11,18,19"
...,...,...,...,...,...,...,...,...,...,...
43186,51.0,technician,married,tertiary,no,825,no,no,very_good,"12,15"
43187,71.0,retired,divorced,primary,no,1729,no,no,very_good,"4,17,19"
43188,72.0,retired,married,secondary,no,5715,no,no,very_good,"5,7,8,9,10,11,13,16,17,18"
43189,57.0,blue-collar,married,secondary,no,668,no,no,very_good,"2,3,7"


### 2) Compute data (dis-)similarity

* To assess the similarity between the customers you could form the dissimilarity matrix for all given attributes. As described in lecture “Measuring Data Similarity”, for every given attribute you first distinguish its type (categorical, ordinal, numerical or set) and then compute the dissimilarity of its values accordingly. For set similarity use the Jaccard similarity between sets. Then, you can calculate the average of the computed dissimilarities to derive the dissimilarity over all attributes. Depending on the machine used to implement this assignment you should decide whether it is feasible to compute the dissimilarity matrices, or, have the computations performed on-the-fly for a pair of customers.

* Creates a function that calculates the dissimilarity between two numerical values.

In [6]:
def num_dissimilarity(x, y, col):
    dis_num = abs(x-y)/(max(col)-min(col))
    return dis_num

* Creates a function that calculates the dissimilarity between two categorical variables.

In [7]:
def categ_dissimilarity(x, y):
    if x == y:
        dis_categ = 0
    else:
        dis_categ = 1
    return dis_categ

* Creates a function that calculates the dissimilarity between two ordinal variables.

In [8]:
def ordinal_dissimilarity(x, y):
    
    if x == 'poor':
        rank_x = 1
    elif x == 'fair':
        rank_x = 2
    elif x == 'good':
        rank_x = 3
    elif x == 'very_good':
        rank_x = 4
    else:
        rank_x = 5
        
    if y == 'poor':
        rank_y = 1
    elif y == 'fair':
        rank_y = 2
    elif y == 'good':
        rank_y = 3
    elif y == 'very_good':
        rank_y = 4
    else:
        rank_y = 5
    
    dis_ord = abs(rank_x - rank_y)/(5-1)
    return dis_ord

* Creates a function that calculates the dissimilarity between two sets.

In [9]:
#define jaccard dissimilarity (size of intersection over size of union)
def jaccard_dissimilarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    dis_jac = 1 - ((float(intersection)) / (union))
    return dis_jac

* Define empty matrices for each variable of the dataset (10 variables) with dimesions 43191x43191.
* Using the constructed functions that calculate the dissimilarities between different pairs of variables, we calculated for each pair of customers the dissimilarities of each variable.
* Fill each matrix with the calculated dissimilarity value between all the pairs of observations for each category.
* Then the average dissimilarity between each pair of customers was calculated and inserted in the final matrix.
* A serious limitation is that this task cannot be completed with the available resources, due to limitations in the availability of main memory.

* <b>Attention:</b> It is advised to not execute the code below due to potential lack of main memory capacity. 

In [92]:
#Define matrices
age_diss_matrix = np.zeros((len(bank), len(bank)))
job_diss_matrix = np.zeros((len(bank), len(bank)))
marital_diss_matrix = np.zeros((len(bank),len(bank)))
education_diss_matrix = np.zeros((len(bank), len(bank)))
default_diss_matrix = np.zeros((len(bank), len(bank)))
balance_diss_matrix = np.zeros((len(bank), len(bank)))
housing_diss_matrix = np.zeros((len(bank), len(bank)))
loan_diss_matrix = np.zeros((len(bank), len(bank)))
rating_diss_matrix = np.zeros((len(bank), len(bank)))
products_diss_matrix = np.zeros((len(bank), len(bank)))


for i in range(len(bank)):
    for j in range(len(bank)):
        a = num_dissimilarity(bank.Age[i], bank.Age[j], bank.Age)
        age_diss_matrix[i, j] = a
        
        b = categ_dissimilarity(bank.Job[i], bank.Job[j])
        job_diss_matrix[i, j] = b
        
        c = categ_dissimilarity(bank.Marital[i], bank.Marital[j])
        marital_diss_matrix[i, j] = c
        
        d = categ_dissimilarity(bank.Education[i], bank.Education[j])
        education_diss_matrix[i, j] = d
        
        e = categ_dissimilarity(bank.Default[i], bank.Default[j])
        default_diss_matrix[i, j] = e
        
        f = num_dissimilarity(bank.Balance[i], bank.Balance[j], bank.Balance)
        balance_diss_matrix[i, j] = f
        
        g = categ_dissimilarity(bank.Housing[i], bank.Housing[j])
        housing_diss_matrix[i, j] = g
        
        h = categ_dissimilarity(bank.Loan[i], bank.Loan[j])
        loan_diss_matrix[i, j] = h
        
        k = ordinal_dissimilarity(bank.Rating[i], bank.Rating[j])
        rating_diss_matrix[i, j] = k
        
        l = jaccard_dissimilarity(bank.Products[i].split(","), bank.Products[j].split(","))
        products_diss_matrix[i, j] = l


dissimilarity_matrix_N = (age_diss_matrix + job_diss_matrix + marital_diss_matrix + 
               education_diss_matrix + default_diss_matrix + balance_diss_matrix + 
               housing_diss_matrix + loan_diss_matrix + rating_diss_matrix + products_diss_matrix)/10

print(dissimilarity_matrix_N)

KeyboardInterrupt: 

* To overcome the previous limitations and evaluate the performance of the code shown above a subset of the dataset was selected that contains only 5 customers.
* As was shown previously, we defined empty matrices for each variable of the dataset (10 variables) with dimesions 5x5.
* Using the constructed functions that calculate the dissimilarities between different pairs of variables, we calculated for each pair of customers the dissimilarities of each variable.
* Then the average dissimilarity between each pair of customers was calculated and inserted in the final dissimilarity matrix.

In [10]:
#Define matrices
age_diss_matrix = np.zeros((5,5))
job_diss_matrix = np.zeros((5,5))
marital_diss_matrix = np.zeros((5,5))
education_diss_matrix = np.zeros((5,5))
default_diss_matrix = np.zeros((5,5))
balance_diss_matrix = np.zeros((5,5))
housing_diss_matrix = np.zeros((5,5))
loan_diss_matrix = np.zeros((5,5))
rating_diss_matrix = np.zeros((5,5))
products_diss_matrix = np.zeros((5,5))


for i in range(5):
    for j in range(5):
        a = num_dissimilarity(bank.Age[i], bank.Age[j], bank.Age)
        age_diss_matrix[i, j] = a
        
        b = categ_dissimilarity(bank.Job[i], bank.Job[j])
        job_diss_matrix[i, j] = b
        
        c = categ_dissimilarity(bank.Marital[i], bank.Marital[j])
        marital_diss_matrix[i, j] = c
        
        d = categ_dissimilarity(bank.Education[i], bank.Education[j])
        education_diss_matrix[i, j] = d
        
        e = categ_dissimilarity(bank.Default[i], bank.Default[j])
        default_diss_matrix[i, j] = e
        
        f = num_dissimilarity(bank.Balance[i], bank.Balance[j], bank.Balance)
        balance_diss_matrix[i, j] = f
        
        g = categ_dissimilarity(bank.Housing[i], bank.Housing[j])
        housing_diss_matrix[i, j] = g
        
        h = categ_dissimilarity(bank.Loan[i], bank.Loan[j])
        loan_diss_matrix[i, j] = h
        
        k = ordinal_dissimilarity(bank.Rating[i], bank.Rating[j])
        rating_diss_matrix[i, j] = k
        
        l = jaccard_dissimilarity(bank.Products[i].split(","), bank.Products[j].split(","))
        products_diss_matrix[i, j] = l
        
dissimilarity_matrix = (age_diss_matrix + job_diss_matrix + marital_diss_matrix + 
               education_diss_matrix + default_diss_matrix + balance_diss_matrix + 
               housing_diss_matrix + loan_diss_matrix + rating_diss_matrix + products_diss_matrix)/10



In [11]:
print(dissimilarity_matrix)

[[0.         0.43851959 0.41912695 0.50668831 0.47348466]
 [0.43851959 0.         0.30798831 0.42429881 0.31885889]
 [0.41912695 0.30798831 0.         0.50170271 0.53487389]
 [0.50668831 0.42429881 0.50170271 0.         0.52088726]
 [0.47348466 0.31885889 0.53487389 0.52088726 0.        ]]


* A function was constructed with the aim of being able to calculate the dissimilarity between a given pair of customers.

In [12]:
#Function dissimilarity between a pair of customers

def pair_dissimilarity(x, y):
    age_diss_matrix = np.zeros((2,2))
    job_diss_matrix = np.zeros((2,2))
    marital_diss_matrix = np.zeros((2,2))
    education_diss_matrix = np.zeros((2,2))
    default_diss_matrix = np.zeros((2,2))
    balance_diss_matrix = np.zeros((2,2))
    housing_diss_matrix = np.zeros((2,2))
    loan_diss_matrix = np.zeros((2,2))
    rating_diss_matrix = np.zeros((2,2))
    products_diss_matrix = np.zeros((2,2))


    for i in range(2):
        for j in range(2):
            if i == j:
                continue
            else: 

                a = num_dissimilarity(bank.Age[x-1], bank.Age[y-1], bank.Age)
                age_diss_matrix[i, j] = a

                b = categ_dissimilarity(bank.Job[x-1], bank.Job[y-1])
                job_diss_matrix[i, j] = b

                c = categ_dissimilarity(bank.Marital[x-1], bank.Marital[y-1])
                marital_diss_matrix[i, j] = c

                d = categ_dissimilarity(bank.Education[x-1], bank.Education[y-1])
                education_diss_matrix[i, j] = d

                e = categ_dissimilarity(bank.Default[x-1], bank.Default[y-1])
                default_diss_matrix[i, j] = e

                f = num_dissimilarity(bank.Balance[x-1], bank.Balance[y-1], bank.Balance)
                balance_diss_matrix[i, j] = f

                g = categ_dissimilarity(bank.Housing[x-1], bank.Housing[y-1])
                housing_diss_matrix[i, j] = g

                h = categ_dissimilarity(bank.Loan[x-1], bank.Loan[y-1])
                loan_diss_matrix[i, j] = h

                k = ordinal_dissimilarity(bank.Rating[x-1], bank.Rating[y-1])
                rating_diss_matrix[i, j] = k

                l = jaccard_dissimilarity(bank.Products[x-1].split(","), bank.Products[y-1].split(","))
                products_diss_matrix[i, j] = l


    dissimilarity_matrix = (age_diss_matrix + job_diss_matrix + marital_diss_matrix + 
                   education_diss_matrix + default_diss_matrix + balance_diss_matrix + 
                   housing_diss_matrix + loan_diss_matrix + rating_diss_matrix + products_diss_matrix)/10

    return dissimilarity_matrix

* Example of the implementation of the function for two random customers.

In [13]:
print(pair_dissimilarity(1,2))

[[0.         0.43851959]
 [0.43851959 0.        ]]



### 3) Nearest Neighbor (NN) search
Using the implementation of the previous step, you will calculate the 10-NN (most similar) customers for the customers with ids listed below (customer id=line number- 1):
1200, 3650, 10400, 14930, 22330, 25671, 29311, 34650, 39200, 42000.
For this task your script must take as input the customer-id and return the list of her
10 nearest neighbors (most similar), along with the corresponding similarity score.

* Create a function `cust_sim_score` that calculates the similarity score between the customer of interest (i.e. one of the 10 customer ids given in this task) and the rest of the customers in the dataset.
* Inside the function an empty DataFrame with three columns ('customer_of_interest', 'customer', 'similarity_score') was constructed.
* The 'for' loop was used to iterate through all the customers in the dataset.
* The dissimilarities of the pairs between the customer of interest and all the customers of the database are calculated.
* Then using the calculated dissimilarities, the similarity of each pair is calculated and inserted into the constructed DataFrame.
* Finally the function returns the constructed DataFrame.

In [15]:
def cust_sim_score(cust):
    
    # create an Empty DataFrame object
    df = pd.DataFrame(columns = ['customer_of_interest','customer', 'similarity_score'])
    
    for j in range(len(bank)):
                if j == int(cust)-1:
                    df = df.append({'customer_of_interest' : cust, 'customer' : j+1, 'similarity_score' : None}, 
                                                           ignore_index=True)
                else: 

                    a = num_dissimilarity(bank.Age[cust-1], bank.Age[j], bank.Age)

                    b = categ_dissimilarity(bank.Job[cust-1], bank.Job[j])

                    c = categ_dissimilarity(bank.Marital[cust-1], bank.Marital[j])

                    d = categ_dissimilarity(bank.Education[cust-1], bank.Education[j])

                    e = categ_dissimilarity(bank.Default[cust-1], bank.Default[j])

                    f = num_dissimilarity(bank.Balance[cust-1], bank.Balance[j], bank.Balance)

                    g = categ_dissimilarity(bank.Housing[cust-1], bank.Housing[j])

                    h = categ_dissimilarity(bank.Loan[cust-1], bank.Loan[j])

                    k = ordinal_dissimilarity(bank.Rating[cust-1], bank.Rating[j])

                    l = jaccard_dissimilarity(bank.Products[cust-1].split(","), bank.Products[j].split(","))

                    dissimilarity_score = (a + b + c + d + e + f + g + h + k + l)/10
                    similarity_score = 1 - dissimilarity_score
                    df = df.append({'customer_of_interest' : cust, 'customer' : j+1, 'similarity_score' : similarity_score} ,
                                                           ignore_index=True)
    return df

* Creates a function that calculates the 10 nearest neighbors of the customer of interest based on the ten highest similarities scores.

In [16]:
def nearest_neighbor(id, df):
    cust_neighbors = df.loc[df.customer_of_interest == id]
    
    ten_super_close_neighbors = cust_neighbors[['customer', 'similarity_score']].sort_values(by='similarity_score', 
                                                                                             ascending=False).iloc[:10]
    
    return ten_super_close_neighbors

* Calculates and prints the 10 nearest neighbors of the selected customer based on the similarity score.

#### 1st Customer: 1200

In [17]:
value_id = 1200

In [18]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 1200 are the following:
       customer  similarity_score
41725   41726.0          0.954603
41663   41664.0          0.926438
23960   23961.0          0.923867
16982   16983.0          0.923040
8755     8756.0          0.920661
4307     4308.0          0.920348
16155   16156.0          0.919752
1105     1106.0          0.917888
13602   13603.0          0.917460
9815     9816.0          0.915249


#### 2nd Customer: 3650

In [19]:
value_id = 3650

In [20]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 3650 are the following:
       customer  similarity_score
6188     6189.0          0.950560
28140   28141.0          0.937784
8066     8067.0          0.936680
6017     6018.0          0.936171
5615     5616.0          0.927949
7306     7307.0          0.923025
20261   20262.0          0.918398
33474   33475.0          0.916447
23346   23347.0          0.913418
37972   37973.0          0.913042


#### 3rd Customer: 10400

In [21]:
value_id = 10400

In [22]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 10400 are the following:
       customer  similarity_score
17597   17598.0          0.964383
20959   20960.0          0.925819
26166   26167.0          0.918353
8652     8653.0          0.915153
28055   28056.0          0.914227
10515   10516.0          0.913437
9096     9097.0          0.911474
15890   15891.0          0.909768
11309   11310.0          0.909424
23686   23687.0          0.897207


#### 4th Customer: 14930

In [23]:
value_id = 14930

In [24]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 14930 are the following:
       customer  similarity_score
33853   33854.0          0.957549
33176   33177.0          0.952029
2705     2706.0          0.938542
8230     8231.0          0.933491
237       238.0          0.932029
31460   31461.0          0.931961
4046     4047.0          0.930308
36759   36760.0          0.929137
4788     4789.0          0.928226
28024   28025.0          0.928155


#### 5th Customer: 22330

In [25]:
value_id = 22330

In [26]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 22330 are the following:
       customer  similarity_score
9839     9840.0          0.931193
17136   17137.0          0.915372
22788   22789.0          0.915206
686       687.0          0.910131
22249   22250.0          0.907943
13616   13617.0          0.900969
21325   21326.0          0.895619
10588   10589.0          0.887767
22614   22615.0          0.884087
22883   22884.0          0.879431


#### 6th Customer: 25671

In [27]:
value_id = 25671

In [28]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 25671 are the following:
       customer  similarity_score
17206   17207.0          0.961472
38035   38036.0          0.935065
23513   23514.0          0.932468
22336   22337.0          0.930909
17776   17777.0          0.922078
27991   27992.0          0.913298
26598   26599.0          0.912098
28053   28054.0          0.912002
41848   41849.0          0.910125
40431   40432.0          0.909962


#### 7th Customer: 29311

In [29]:
value_id = 29311

In [30]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 29311 are the following:
       customer  similarity_score
24619   24620.0          0.835948
23959   23960.0          0.833641
34476   34477.0          0.826136
34630   34631.0          0.823541
3145     3146.0          0.822697
13990   13991.0          0.822025
7171     7172.0          0.818546
6764     6765.0          0.818267
25207   25208.0          0.817017
36153   36154.0          0.816659


#### 8th Customer: 34650

In [31]:
value_id = 34650

In [32]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 34650 are the following:
       customer  similarity_score
6901     6902.0          0.931291
27642   27643.0          0.927901
20566   20567.0          0.924065
7657     7658.0          0.923906
30714   30715.0          0.922335
33301   33302.0          0.921600
31571   31572.0          0.921122
32192   32193.0          0.920330
24951   24952.0          0.920001
36354   36355.0          0.918987


#### 9th Customer: 39200

In [33]:
value_id = 39200

In [34]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 39200 are the following:
       customer  similarity_score
13170   13171.0          0.958222
29358   29359.0          0.929250
37140   37141.0          0.928357
13046   13047.0          0.922020
9743     9744.0          0.921712
29462   29463.0          0.921251
27082   27083.0          0.920309
27578   27579.0          0.917305
42808   42809.0          0.912519
29054   29055.0          0.912070


#### 10th Customer: 42000

In [35]:
value_id = 42000

In [36]:
customers_q3 = cust_sim_score(int(value_id))
top_nn = nearest_neighbor(int(value_id), customers_q3)
print("The 10 Nearest Neighbors of customer {} are the following:".format(value_id))
print(top_nn)

The 10 Nearest Neighbors of customer 42000 are the following:
       customer  similarity_score
32470   32471.0          0.958657
28026   28027.0          0.950954
31800   31801.0          0.943429
41315   41316.0          0.942950
39517   39518.0          0.937329
22741   22742.0          0.935395
41143   41144.0          0.935216
42418   42419.0          0.935024
32492   32493.0          0.933916
39424   39425.0          0.933633


### 4) Customer rating prediction
For this assignment you will implement a classification algorithm which, for a given
customer, will predict his rating (poor, fair, good, very good, excellent) for the bank.
In order to implement the classification for a given customer you need to:
1. Calculate the similarities between the given customer and all other customers and compute his 10-nn (most similar) customers. <b>IMPORTANT:</b> In the similarity calculations for this step you need to exclude the customer rating attribute.
2. Based only on the 10 most similar customers computed in the previous step, predict the customer rating rank using:
> * The average rating rank of the 10 most similar customers (rounded to the nearest integer).
> * The weighted average rating rank of the 10 most similar customers (rounded to the nearest integer).
---  
* Where:
 * rating(i) = the rating of the i-th nearest neighbor (i=1 for the most similar customer)
 * similarity(i) = the similarity of the i-th nearest neighbor with the given customer

3. For the evaluation of your classification algorithm you will use the 50 first records of the bank dataset and predict the rating for them. Then, for all n=50 records calculate the Mean Prediction Error for both prediction methods. Mean Prediction Error

---
#### Question 4.1

* Select a subset of the dataset that contains all the columns except the one referring to `Rating`.

In [37]:
mini_bank = bank[['Age', 'Job', 'Marital', 'Education', 'Default', 'Balance', 'Housing', 'Loan', 'Products']]
mini_bank

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Products
0,33.0,entrepreneur,married,secondary,no,2,yes,yes,"1,3,16,17,19"
1,35.0,management,married,tertiary,no,231,yes,no,"4,8,16"
2,41.0,management,single,tertiary,no,447,yes,yes,"7,16"
3,42.0,entrepreneur,divorced,tertiary,yes,2,yes,no,"1,3,8,10,11,12,18,19"
4,58.0,retired,married,primary,no,121,yes,no,"4,5,6,7,11,18,19"
...,...,...,...,...,...,...,...,...,...
43186,51.0,technician,married,tertiary,no,825,no,no,"12,15"
43187,71.0,retired,divorced,primary,no,1729,no,no,"4,17,19"
43188,72.0,retired,married,secondary,no,5715,no,no,"5,7,8,9,10,11,13,16,17,18"
43189,57.0,blue-collar,married,secondary,no,668,no,no,"2,3,7"


* Asks user to input a customer of interest.

In [39]:
cust_id = input("Please enter the id number of the customer of interest: \n")

Please enter the id number of the customer of interest: 
151


* Create a function `cust_sim_score_v2` that calculates the similarity score between the customer of interest ( i.e. the customer given from the user ) and the rest customers in the dataset.
* The main difference between functions `cust_sim_score` and `cust_sim_score_v2` is that the latter does not take into account the column `Rating`.
* Inside the function an an empty DataFrame with three columns ('customer_of_interest', 'customer', 'similarity_score') was constructed.
* The 'for' loop was used to iterate through all the customers in the dataset.
* The dissimilarities of pairs between the customer of interest ( i.e. the customer given from the user ) and all the customers of the database are calculated, without taking into account the `Rating` column.
* Then using the calculated dissimilarities, the similarity of each pair is calculated and inserted into the constructed DataFrame.
* Finally the function returns the constructed DataFrame.

In [40]:
def cust_sim_score_v2(cust):
    
    # create an Empty DataFrame object
    df = pd.DataFrame(columns = ['customer_of_interest','customer', 'similarity_score'])
    
    for j in range(len(mini_bank)):
                if j == int(cust)-1:
                    df = df.append({'customer_of_interest' : cust, 'customer' : j+1, 'similarity_score' : None}, 
                                                           ignore_index=True)
                else: 

                    a = num_dissimilarity(bank.Age[cust-1], bank.Age[j], bank.Age)

                    b = categ_dissimilarity(bank.Job[cust-1], bank.Job[j])

                    c = categ_dissimilarity(bank.Marital[cust-1], bank.Marital[j])

                    d = categ_dissimilarity(bank.Education[cust-1], bank.Education[j])

                    e = categ_dissimilarity(bank.Default[cust-1], bank.Default[j])

                    f = num_dissimilarity(bank.Balance[cust-1], bank.Balance[j], bank.Balance)

                    g = categ_dissimilarity(bank.Housing[cust-1], bank.Housing[j])

                    h = categ_dissimilarity(bank.Loan[cust-1], bank.Loan[j])

                    l = jaccard_dissimilarity(bank.Products[cust-1].split(","), bank.Products[j].split(","))

                    dissimilarity_score = (a + b + c + d + e + f + g + h + l)/9
                    similarity_score = 1 - dissimilarity_score
                    df = df.append({'customer_of_interest' : cust, 'customer' : j+1, 'similarity_score' : similarity_score} ,
                                                           ignore_index=True)
    return df

* Calculates the ten nearest neighbors of the customer given by the user based on the similarities values.

In [41]:
customers_q4 = cust_sim_score_v2(int(cust_id))
top_nn_q4 = nearest_neighbor(int(cust_id), customers_q4)
top_nn_q4

,customer,similarity_score
19082,19083.0,0.952856
2946,2947.0,0.949174
9343,9344.0,0.947142
18107,18108.0,0.943402
20161,20162.0,0.942132
10034,10035.0,0.941000
11827,11828.0,0.937824
19421,19422.0,0.937263
22824,22825.0,0.937107
12350,12351.0,0.937091


---
#### Question 4.2

* Copied the DataFrame and assigned to a different variable.

In [42]:
#take ratings
top_nn_data = copy.deepcopy(bank.iloc[top_nn_q4.customer.index])
top_nn_data

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
19082,52.0,blue-collar,married,primary,no,2569,no,no,very_good,"3,5,9,11,15,20"
2946,49.0,blue-collar,married,primary,no,0,no,no,good,"3,7,9,11,15,18,20"
9343,49.0,blue-collar,married,primary,no,902,no,no,very_good,"3,4,5,9,11,14,15,16,17,18"
18107,55.0,blue-collar,married,primary,no,0,no,no,good,"3,5,7,11,14,16,18"
20161,45.0,blue-collar,married,primary,no,147,no,no,very_good,"1,2,3,7,9,11,12,14,15,16"
10034,48.0,blue-collar,married,primary,no,554,no,no,very_good,"3,5,10,11,14,15,18,20"
11827,49.0,blue-collar,married,primary,no,126,no,no,very_good,"2,3,9,10,11,14,15,17,18"
19421,57.0,blue-collar,married,primary,no,2112,no,no,very_good,"3,4,5,6,9,11,14,15,17,18"
22824,45.0,blue-collar,married,primary,no,121,no,no,very_good,"5,8,9,11,14,15,16,17"
12350,55.0,blue-collar,married,primary,no,137,no,no,very_good,"5,9,14,15,16"


* Changes the values of the column `Rating` (ordinal variable) based on their perceived rank (1 to 5).

In [43]:
top_nn_data.Rating[top_nn_data.Rating=='poor'] = 1
top_nn_data.Rating[top_nn_data.Rating=='fair'] = 2
top_nn_data.Rating[top_nn_data.Rating=='good'] = 3
top_nn_data.Rating[top_nn_data.Rating=='very_good'] = 4
top_nn_data.Rating[top_nn_data.Rating=='excellent'] = 5
top_nn_data

<ipython-input-43-bbbfc6fa8360>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_nn_data.Rating[top_nn_data.Rating=='poor'] = 1
<ipython-input-43-bbbfc6fa8360>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_nn_data.Rating[top_nn_data.Rating=='fair'] = 2
<ipython-input-43-bbbfc6fa8360>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_nn_data.Rating[top_nn_data.Rating=='good'] = 3
<ipython-input-43-bbbfc6fa8360>:4: SettingW

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
19082,52.0,blue-collar,married,primary,no,2569,no,no,4,"3,5,9,11,15,20"
2946,49.0,blue-collar,married,primary,no,0,no,no,3,"3,7,9,11,15,18,20"
9343,49.0,blue-collar,married,primary,no,902,no,no,4,"3,4,5,9,11,14,15,16,17,18"
18107,55.0,blue-collar,married,primary,no,0,no,no,3,"3,5,7,11,14,16,18"
20161,45.0,blue-collar,married,primary,no,147,no,no,4,"1,2,3,7,9,11,12,14,15,16"
10034,48.0,blue-collar,married,primary,no,554,no,no,4,"3,5,10,11,14,15,18,20"
11827,49.0,blue-collar,married,primary,no,126,no,no,4,"2,3,9,10,11,14,15,17,18"
19421,57.0,blue-collar,married,primary,no,2112,no,no,4,"3,4,5,6,9,11,14,15,17,18"
22824,45.0,blue-collar,married,primary,no,121,no,no,4,"5,8,9,11,14,15,16,17"
12350,55.0,blue-collar,married,primary,no,137,no,no,4,"5,9,14,15,16"


* Calculate the average rating rank of the given customer based on the `Rating` of his 10 closest neighbors.

In [44]:
average_rating = round(top_nn_data.Rating.mean())
average_rating

4

* Add the `Rating` column to the constructed DataFrame.

In [45]:
top_nn_q4['Rank'] = list(top_nn_data['Rating'])
top_nn_q4

,customer,similarity_score,Rank
19082,19083.0,0.952856,4
2946,2947.0,0.949174,3
9343,9344.0,0.947142,4
18107,18108.0,0.943402,3
20161,20162.0,0.942132,4
10034,10035.0,0.941000,4
11827,11828.0,0.937824,4
19421,19422.0,0.937263,4
22824,22825.0,0.937107,4
12350,12351.0,0.937091,4


* Calculate the Weighted Average Rating Rank of the given customer based on the similarity and rank of the 10 closest neighbors.

In [46]:
weighted_average = round(sum((top_nn_q4.similarity_score * top_nn_q4.Rank))/sum(top_nn_q4.similarity_score))
weighted_average

4

In [47]:
print("The Rating that was predicted for the customer of interest based on the average rating method is: {}".format(average_rating))
print("The Rating that was predicted for the customer of interest based on the weighted average rating method is: {}".format(weighted_average))

The Rating that was predicted for the customer of interest based on the average rating method is: 4
The Rating that was predicted for the customer of interest based on the weighted average rating method is: 4


---
#### Question 4.3

* Copied the DataFrame with the first 50 customers and assigned to a different variable.

In [48]:
bank_q4_3 = copy.deepcopy(bank.iloc[0:50])
bank_q4_3

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
0,33.0,entrepreneur,married,secondary,no,2,yes,yes,poor,"1,3,16,17,19"
1,35.0,management,married,tertiary,no,231,yes,no,good,"4,8,16"
2,41.0,management,single,tertiary,no,447,yes,yes,fair,"7,16"
3,42.0,entrepreneur,divorced,tertiary,yes,2,yes,no,fair,"1,3,8,10,11,12,18,19"
4,58.0,retired,married,primary,no,121,yes,no,good,"4,5,6,7,11,18,19"
5,43.0,technician,single,secondary,no,593,yes,no,good,"1,5,7,8,11,16,17,19,20"
6,41.0,admin.,divorced,secondary,no,270,yes,no,good,"2,3,4,5,9,10,11,12,14,17"
7,29.0,admin.,single,secondary,no,390,yes,no,good,17
8,53.0,technician,married,secondary,no,6,yes,no,good,"8,13,14,16,17,18,19,20"
9,41.0,services,married,secondary,no,162,yes,no,good,3


* Changes the values of the column `Rating` (ordinal variable) based on their perceived rank (1 to 5).

In [49]:
bank_q4_3.Rating[bank_q4_3.Rating=='poor'] = 1
bank_q4_3.Rating[bank_q4_3.Rating=='fair'] = 2
bank_q4_3.Rating[bank_q4_3.Rating=='good'] = 3
bank_q4_3.Rating[bank_q4_3.Rating=='very_good'] = 4
bank_q4_3.Rating[bank_q4_3.Rating=='excellent'] = 5
bank_q4_3

<ipython-input-49-528cf8d5d5a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_q4_3.Rating[bank_q4_3.Rating=='poor'] = 1
<ipython-input-49-528cf8d5d5a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_q4_3.Rating[bank_q4_3.Rating=='fair'] = 2
<ipython-input-49-528cf8d5d5a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_q4_3.Rating[bank_q4_3.Rating=='good'] = 3
<ipython-input-49-528cf8d5d5a9>:4: SettingWithCopyWarni

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
0,33.0,entrepreneur,married,secondary,no,2,yes,yes,1,"1,3,16,17,19"
1,35.0,management,married,tertiary,no,231,yes,no,3,"4,8,16"
2,41.0,management,single,tertiary,no,447,yes,yes,2,"7,16"
3,42.0,entrepreneur,divorced,tertiary,yes,2,yes,no,2,"1,3,8,10,11,12,18,19"
4,58.0,retired,married,primary,no,121,yes,no,3,"4,5,6,7,11,18,19"
5,43.0,technician,single,secondary,no,593,yes,no,3,"1,5,7,8,11,16,17,19,20"
6,41.0,admin.,divorced,secondary,no,270,yes,no,3,"2,3,4,5,9,10,11,12,14,17"
7,29.0,admin.,single,secondary,no,390,yes,no,3,17
8,53.0,technician,married,secondary,no,6,yes,no,3,"8,13,14,16,17,18,19,20"
9,41.0,services,married,secondary,no,162,yes,no,3,3


* Construction of an empty Dataframe with three columns (`customer_of_interest`, `customer`, `similarity_score`)​.

In [50]:
# create an Empty DataFrame object
customers_50 = pd.DataFrame(columns = ['customer_of_interest','customer', 'similarity_score'])

print(customers_50)

Empty DataFrame
Columns: [customer_of_interest, customer, similarity_score]
Index: []


* The 2 'for' loops were used in order to iterate through all the customers in the dataset.
* The dissimilarities of the pairs between the customer of interest and all the customers of the database are calculated, without taking into account the `Rating` column.
* Then using the calculated dissimilarities, the similarity of each pair is calculated and inserted into the constructed matrix.

In [51]:
for i in range(len(bank_q4_3)):
    for j in range(len(bank_q4_3)):
                if j == i:
                    customers_50 = customers_50.append({'customer_of_interest' : i, 'customer' : j, 'similarity_score' : None, 'Rank' : None}, 
                                                       ignore_index=True)
                else: 

                    a = num_dissimilarity(bank_q4_3.Age[i], bank_q4_3.Age[j], bank_q4_3.Age)

                    b = categ_dissimilarity(bank_q4_3.Job[i], bank_q4_3.Job[j])

                    c = categ_dissimilarity(bank_q4_3.Marital[i], bank_q4_3.Marital[j])

                    d = categ_dissimilarity(bank_q4_3.Education[i], bank_q4_3.Education[j])

                    e = categ_dissimilarity(bank_q4_3.Default[i], bank_q4_3.Default[j])

                    f = num_dissimilarity(bank_q4_3.Balance[i], bank_q4_3.Balance[j], bank_q4_3.Balance)

                    g = categ_dissimilarity(bank_q4_3.Housing[i], bank_q4_3.Housing[j])

                    h = categ_dissimilarity(bank_q4_3.Loan[i], bank_q4_3.Loan[j])

                    l = jaccard_dissimilarity(bank_q4_3.Products[i].split(","), bank_q4_3.Products[j].split(","))
                    
                    dissimilarity_score_q4 = (a + b + c + d + e + f + g + h + l)/9
                    similarity_score_q4 = 1 - dissimilarity_score_q4
                    customers_50 = customers_50.append({'customer_of_interest' : i, 'customer' : j, 'similarity_score' : similarity_score_q4 ,
                                                       'Rank' : bank_q4_3.Rating[j]} , ignore_index=True)



* Prints the constructed DataFrame.

In [52]:
customers_50

,customer_of_interest,customer,similarity_score,Rank
0,0.0,0.0,NaN,NaN
1,0.0,1.0,0.562944,3.0
2,0.0,2.0,0.544891,2.0
3,0.0,3.0,0.450000,2.0
4,0.0,4.0,0.487295,3.0
...,...,...,...,...
2495,49.0,45.0,0.552981,2.0
2496,49.0,46.0,0.398388,1.0
2497,49.0,47.0,0.307996,3.0
2498,49.0,48.0,0.560137,3.0


* Performs a GroupBy in the constructed dataframe based on the `customer_of_interest`.
* Additionally the results are sorted in decreasing order based on  the similarity_score of each pair of customers.
* Therefore, the 10 closest neighbors for each customer of interest (0-49) are selected.

In [53]:
customers_grouped = customers_50.groupby(by='customer_of_interest', as_index=False)['customer', 
                        'similarity_score', 'Rank'].apply(lambda x: x.nlargest(10, 'similarity_score'))

customers_grouped

<ipython-input-53-a225e1e54f75>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  customers_grouped = customers_50.groupby(by='customer_of_interest', as_index=False)['customer',


customer  similarity_score  Rank
0  21        21.0          0.847645   2.0
   14        14.0          0.765168   2.0
   26        26.0          0.734475   2.0
   39        39.0          0.682079   2.0
   13        13.0          0.678992   3.0
...           ...               ...   ...
49 2454       4.0          0.751266   3.0
   2462      12.0          0.733367   3.0
   2489      39.0          0.695724   2.0
   2469      19.0          0.679623   3.0
   2475      25.0          0.677621   3.0

[500 rows x 3 columns]

* Perform a tuple unpacking of the index of the dataframe and appends the customer id of each customer to the list.

In [54]:
usefull_index = []
for i, j in customers_grouped.index:
    usefull_index.append(i)


* Adds to the dataframe a customer of interest column based on the values extracted in the previous block of code.

In [55]:
customers_grouped['customer_of_interest'] = usefull_index
customers_grouped = customers_grouped[['customer_of_interest', 'customer', 'similarity_score', 'Rank']]
customers_grouped

customer_of_interest  customer  similarity_score  Rank
0  21                       0      21.0          0.847645   2.0
   14                       0      14.0          0.765168   2.0
   26                       0      26.0          0.734475   2.0
   39                       0      39.0          0.682079   2.0
   13                       0      13.0          0.678992   3.0
...                       ...       ...               ...   ...
49 2454                    49       4.0          0.751266   3.0
   2462                    49      12.0          0.733367   3.0
   2489                    49      39.0          0.695724   2.0
   2469                    49      19.0          0.679623   3.0
   2475                    49      25.0          0.677621   3.0

[500 rows x 4 columns]

* Reset the index of the DataFrame.

In [56]:
customers_grouped = customers_grouped.reset_index()

* Removes unnecessary columns from the DataFrame.

In [57]:
del(customers_grouped['level_0'])
del(customers_grouped['level_1'])

customers_grouped

,customer_of_interest,customer,similarity_score,Rank
0,0,21.0,0.847645,2.0
1,0,14.0,0.765168,2.0
2,0,26.0,0.734475,2.0
3,0,39.0,0.682079,2.0
4,0,13.0,0.678992,3.0
...,...,...,...,...
495,49,4.0,0.751266,3.0
496,49,12.0,0.733367,3.0
497,49,39.0,0.695724,2.0
498,49,19.0,0.679623,3.0


* Perform a GroupBy on the `customer_of_interest` on the constructed DataFrame and calculate the average rank for each customer. 

In [58]:
cust_class_rank = customers_grouped.groupby(by = 'customer_of_interest')['Rank'].mean().round()
cust_class_rank

customer_of_interest
0     2.0
1     3.0
2     3.0
3     3.0
4     3.0
5     2.0
6     3.0
7     2.0
8     3.0
9     3.0
10    3.0
11    3.0
12    3.0
13    3.0
14    2.0
15    3.0
16    2.0
17    3.0
18    2.0
19    3.0
20    2.0
21    2.0
22    3.0
23    2.0
24    2.0
25    3.0
26    2.0
27    3.0
28    3.0
29    3.0
30    3.0
31    3.0
32    3.0
33    3.0
34    3.0
35    3.0
36    3.0
37    3.0
38    3.0
39    3.0
40    3.0
41    2.0
42    3.0
43    3.0
44    3.0
45    3.0
46    2.0
47    2.0
48    2.0
49    3.0
Name: Rank, dtype: float64

* Creation of the final DataFrame that will be used to store the classified ranks of the customers of interest.

In [59]:
classification_table = pd.DataFrame({'customer':cust_class_rank.index, 'Average_Rating':cust_class_rank.values})
classification_table

,customer,Average_Rating
0,0,2.0
1,1,3.0
2,2,3.0
3,3,3.0
4,4,3.0
5,5,2.0
6,6,3.0
7,7,2.0
8,8,3.0
9,9,3.0


* Creation of an Empty List.

In [60]:
#Empty List

average_weighted_rank = []

* Iterate through all the unique set of customers of interest in the dataset (0-49).
* Iterate for each customer of interest through his 10 closest neighbors.
* Calculate the Weighted Average Rating Rank for each one of the customers of interest.

In [61]:
for i in set(customers_grouped.customer_of_interest):
    numerator = 0
    for j in list(customers_grouped.customer[customers_grouped.customer_of_interest==i]):
        sim = float(customers_grouped.similarity_score[(customers_grouped.customer_of_interest==i) & (customers_grouped.customer==j)].values) 
        rank = float(customers_grouped.Rank[(customers_grouped.customer_of_interest==i) & (customers_grouped.customer==j)].values) 
        numerator = numerator + sim*rank

    denominator = float(sum(customers_grouped.similarity_score[customers_grouped.customer_of_interest==i].values))
    result = round(numerator/denominator)
    average_weighted_rank.append(result)
        

* Add to the final DataFrame the calculated Average Rating Rank for all the customers of interest.

In [62]:
classification_table['Average_Weighted_Rating'] = average_weighted_rank
classification_table

,customer,Average_Rating,Average_Weighted_Rating
0,0,2.0,2
1,1,3.0,3
2,2,3.0,3
3,3,3.0,3
4,4,3.0,3
5,5,2.0,2
6,6,3.0,3
7,7,2.0,2
8,8,3.0,3
9,9,3.0,3


* Add to the Final DataFrame the actual Rating of each customer of interest.

In [63]:
classification_table['Actual_Rating'] = bank_q4_3.Rating
classification_table

,customer,Average_Rating,Average_Weighted_Rating,Actual_Rating
0,0,2.0,2,1
1,1,3.0,3,3
2,2,3.0,3,2
3,3,3.0,3,2
4,4,3.0,3,3
5,5,2.0,2,3
6,6,3.0,3,3
7,7,2.0,2,3
8,8,3.0,3,3
9,9,3.0,3,3


* Calculates the Mean Prediction Error for the two methods that were used to classify the customers of interest.
* Presents the results.

In [64]:
mpe_avg_rat = sum(abs(classification_table.Average_Rating - classification_table.Actual_Rating))/50
mpe_avg_weigh_rat = sum(abs(classification_table.Average_Weighted_Rating - classification_table.Actual_Rating))/50

In [65]:
#Empty Datframe
mpe_table = pd.DataFrame({'Index': range(1), 'MPE_Average':mpe_avg_rat, 'MPE_Weighted_Average':mpe_avg_weigh_rat})
del(mpe_table['Index'])
mpe_table

,MPE_Average,MPE_Weighted_Average
0,0.42,0.46


* It can be observed that the simple average method based on the 10 most similar neighbors of each customer of interest performs slighlty better in the final results, compared to Weighted Average Method that was also used.